<a href="https://colab.research.google.com/github/Bpriya42/bias-reduction-facial-recognition/blob/main/Facial_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

1. Install the Seldonian Engine
2. Mout the drive
3. Install Seldonian experiments
4. Load datasets
5. Complete imports

In [ ]:
! pip install --upgrade seldonian-engine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install git
!git clone https://github.com/seldonian-toolkit/Experiments.git "/content/drive/My Drive/ColabNotebooks/facial_recog_racial_bias_mitigation/seldonian_experiment"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into '/content/drive/My Drive/ColabNotebooks/facial_recog_racial_bias_mitigation/seldonian_experiment'...
remote: Enumerating objects: 3528, done.
remote: Counting objects: 100% (640/640), done.
remote: Compressing objects: 100% (308/308), done.
remote: Total 3528 (delta 487), reused 458 (delta 332), pack-reused 2888 (from 1)
Receiving objects: 100% (3528/3528), 15.19 MiB | 7.15 MiB/s, done.
Resolving deltas: 100% (2690/2690), done.
Updating files: 100% (101/101), done.


In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.29.0 requires matplotlib>=3.7.1, but you have matplotlib 3.5.1 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Import PyTorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt
import numpy as np

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

PyTorch version: 2.5.1+cu121
torchvision version: 0.20.1+cu121


# Preparing Data


In [ ]:
from helper import *

In [ ]:
# Load and combine the dataset
df_train, df_test = load_fair_dataset_combined()

df_train

,image,age,gender,race
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,6,0,0
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,4,1,1
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1,1,2
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3,1,1
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3,1,1
...,...,...,...,...
86739,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3,0,1
86740,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,2,0,1
86741,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,8,1,1
86742,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,2,1,2


In [ ]:
from torch.utils.data import DataLoader

train_dataset = BaselineDataset(df_train['image'], df_train['gender'], transform=transform)
test_dataset = BaselineDataset(df_train['image'], df_train['gender'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Baseline Models